# Libraries

In [109]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

# Network
import torch
import torch.nn as nn
import torch.nn.functional as F

# Utility functions
from utility import truncated_poisson_sampling, create_item_pool, create_agent_utility, rewards_func_test

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parameters

In [110]:
# Game setup
num_agents = 2         # Number of agents playing the game
num_types = 3          # Number of item types
max_item = 5           # Maximum number of each item in a pool
max_utility = 10       # Maximum utility value for agents

# Turn sampling
lam = 7                # Poisson parameter
max_N = 10             # Maximum number of turns
min_N = 4              # Minimum number of turns

# Linguistic channel
num_vocab = 10         # Symbol vocabulary size for linguistic channel
len_message = 6        # Linguistic message length

# Testing
N_ep = 20              # Number of episodes
num_games = 128        # Number of games per episode (batch size)

# Network (copied from Untitled-Batch)

In [111]:
class combined_policy(nn.Module):
    def __init__(self, embedding_dim = 100, batch_size = 128, num_layers = 1, bias = True, batch_first = False, dropout = 0, bidirectional = False):
        super(combined_policy, self).__init__()
        # Save variables
        self.embedding_dim = embedding_dim # Hidden layer dimensions
        self.batch_size = batch_size       # Batch size (updated every forward pass)
        self.log_p = torch.zeros([batch_size,1])                     # Store policy log likelihood for REINFORCE
        
        # Encoding -------------------------------------------------------------
        
        # Numerical encoder
        self.encoder1 = nn.Embedding(max_utility+1, embedding_dim)
        # Linguistic encoder
        self.encoder2 = nn.Embedding(num_vocab+1, embedding_dim)
        
        # Item context LSTM
        self.lstm1 = nn.LSTM(embedding_dim, embedding_dim, num_layers, bias, batch_first, dropout, bidirectional)
        # Linguistic LSTM
        self.lstm2 = nn.LSTM(embedding_dim, embedding_dim, num_layers, bias, batch_first, dropout, bidirectional)
        # Proposal LSTM
        self.lstm3 = nn.LSTM(embedding_dim, embedding_dim, num_layers, bias, batch_first, dropout, bidirectional)
        
        # Outputs of the 3 LSTMS get concatenated together
        
        # Feed-forward
        self.ff = nn.Linear(3*embedding_dim, embedding_dim)
        
        # Output of feed-forward is the input for the policy networks
        
        # Policy ---------------------------------------------------------------
        
        # Termination policy
        self.policy_term = nn.Linear(embedding_dim, 1)
        # Linguistic policy
        self.policy_ling = nn.LSTM(embedding_dim, embedding_dim, num_layers, bias, batch_first, dropout, bidirectional)
        self.ff_ling = nn.Linear(embedding_dim, num_vocab)
        # Proposal policies
        self.policy_prop = []
        for i in range(num_types):
            ff = nn.Linear(embedding_dim, max_item+1)
            self.policy_prop.append(ff)
        
    def forward(self, x, test, batch_size=128):
        # Inputs --------------------------------------------------------------------
        # x = list of three elements consisting of:
        #   1. item context (longtensor of shape batch_size x (2*num_types))
        #   2. previous linguistic message (longtensor of shape batch_size x len_message)
        #   3. previous proposal (longtensor of shape batch_size x num_types)
        # test = whether training or testing (testing selects actions greedily)
        # batch_size = batch size
        # Outputs -------------------------------------------------------------------
        # term = binary variable where 1 indicates proposal accepted => game finished (longtensor of shape batch_size x 1)
        # message = crafted linguistic message (longtensor of shape batch_size x len_message)
        # prop = crafted proposal (longtensor of shape batch_size x num_types)
        # entropy_loss = Number containing the sum of policy entropies (should be total entropy by additivity)
        
        # Update batch_size variable (changes throughout training due to sieving (see survivors below))
        self.batch_size = batch_size
        
        # Extract inputs ------------------------------------------------------------
        
        # Item context
        x1 = x[0]
        # Previous linguistic message
        x2 = x[1]
        # Previous proposal
        x3 = x[2]  

        # Encoding ------------------------------------------------------------------

        # Initial embedding
        x1 = self.encoder1(x1).transpose(0,1)
        x2 = self.encoder2(x2).transpose(0,1)
        x3 = self.encoder1(x3).transpose(0,1) # Same encoder as item context
            
        # LSTM for item context
        h1 = torch.zeros(1,self.batch_size,self.embedding_dim) # Initial hidden
        c1 = torch.zeros(1,self.batch_size,self.embedding_dim) # Initial cell

        for i in range(x1.size()[0]):
            _, (h1,c1) = self.lstm1(x1[i].view(1,self.batch_size,self.embedding_dim),(h1,c1))
        x1_encoded = h1
        
        # LSTM for linguistic
        h2 = torch.zeros(1,self.batch_size,self.embedding_dim) # Initial hidden
        c2 = torch.zeros(1,self.batch_size,self.embedding_dim) # Initial cell

        for i in range(x2.size()[0]):
            _, (h2,c2) = self.lstm2(x2[i].view(1,self.batch_size,self.embedding_dim),(h2,c2))
        x2_encoded = h2
        
        # LSTM for proposal
        h3 = torch.zeros(1,self.batch_size,self.embedding_dim) # Initial hidden
        c3 = torch.zeros(1,self.batch_size,self.embedding_dim) # Initial cell

        for i in range(x3.size()[0]):
            _, (h3,c3) = self.lstm2(x3[i].view(1,self.batch_size,self.embedding_dim),(h3,c3))
        x3_encoded = h3

        # Concatenate side-by-side
        h = torch.cat([x1_encoded,x2_encoded,x3_encoded],2)

        # Feedforward
        h = self.ff(h)
        h = F.relu(h) # Hidden layer input for policy networks
        
        # Policy ------------------------------------------------------------------

        # Termination -----------------------------------------------
        p_term = F.sigmoid(self.policy_term(h)).view(self.batch_size,1)

        # Entropy
        entropy_term = -(p_term * p_term.log()) - (torch.ones(self.batch_size,1)-p_term * (torch.ones(self.batch_size,1)-p_term.log()))
        entropy_term = torch.sum(entropy_term)
        
        if test:
            # Greedy
            term = torch.round(p_term).long()
        else:
            # Sample
            term = torch.bernoulli(p_term).long()
            
        # log p for REINFORCE
        log_p_term = torch.zeros(self.batch_size,1)
        log_p_term = (term.float() * p_term + (torch.ones(self.batch_size,1)-term.float()) * (torch.ones(self.batch_size,1)-p_term)).log2()
        
        # Linguistic construction ----------------------------------
        h_ling = h.clone() # Initial hidden state
        c_ling = torch.zeros(1,self.batch_size,self.embedding_dim) # Initial cell state
        letter = torch.zeros(self.batch_size,1).long() # Initial letter (dummy)
        entropy_letter = torch.zeros([self.batch_size,len_message])
        
        # log p for REINFORCE 
        log_p_letter = torch.zeros([self.batch_size,1])

        message = torch.zeros(self.batch_size,len_message) # Message
        for i in range(len_message):
            embedded_letter = self.encoder2(letter)

            _, (h_ling,c_ling) = self.policy_ling(embedded_letter.view(1,self.batch_size,self.embedding_dim),(h_ling,c_ling))
            logit = self.ff_ling(h_ling)
            p_letter = F.softmax(logit,dim=2).view(self.batch_size,num_vocab)

            entropy_letter[:,i] = -torch.sum(p_letter*p_letter.log2(),1)

            if test:
                # Greedy
                letter = p_letter.argmax(dim=1).view(self.batch_size,1).long()
            else:
                # Sample
                letter = torch.multinomial(p_letter,1).long()
                
            # Gather the probabilities for the letters we've picked
            probs = torch.gather(p_letter, 1, letter)
            log_p_letter = log_p_letter + probs.log2()
                
            message[:,i] = letter.squeeze()
            
        message = message.long()
        entropy_letter = torch.sum(entropy_letter)     
   
        # Proposal ----------------------------------------------
        p_prop = []
        prop = []
        
        #prop = torch.zeros([self.batch_size,num_types]).long()
        entropy_prop_list = [0,0,0]
        
        # log p for REINFORCE 
        log_p_prop = torch.zeros([self.batch_size,1])

        for i in range(num_types):
            p_prop.append(F.sigmoid(self.policy_prop[i](h)))
            
            entropy_prop_list[i] = -torch.sum(p_prop[i]*p_prop[i].log())
            
            p_prop[i] = p_prop[i].view(self.batch_size,max_item+1)

            if test:
                # Greedy
                #prop[:,i] = p_prop[i].argmax(dim=1)
                prop.append(p_prop[i].argmax(dim=1))
            else:
                # Sample
                #prop[:,i] = torch.multinomial(p_prop,1)
                prop.append(torch.multinomial(p_prop,1))
                
            # Gather the probabilities for the letters we've picked
            probs = torch.gather(p_prop[i], 1, prop[i].view(self.batch_size,1))
            log_p_prop = log_p_prop + probs.log2()
              
        prop = torch.stack(prop).transpose(0,1)
        
        return (term,message,prop)

# Test

In [118]:
# Random seeds for testing
torch.manual_seed(0)
np.random.seed(0)

# Load trained agents
Agents = []
for i in range(num_agents):
    Agent = combined_policy()
    #Agent.load_state_dict(torch.load('saved_model_agent_' + str(i) + '.pt'))
    Agents.append(Agent)
    
average_rewards = [torch.zeros(N_ep), torch.zeros(N_ep)]       # Average rewards for each game

print('Start ----------------')
# Loop over episodes
for i_ep in range(N_ep):
    # Setting up games -----------------------------------------------------------------------
    # Game setup
    # Truncated Poisson sampling for number of turns in each game
    N = truncated_poisson_sampling(lam, min_N, max_N, num_games)
    # Item pools for each game
    pool = create_item_pool(num_types, max_item, num_games)
    # Item contexts for each game
    item_contexts = [] # Each agent has different utilities (but same pool)
    for i in range(num_agents):
        utility = create_agent_utility(num_types, max_utility, num_games)
        item_contexts.append(torch.cat([pool, utility],1))
    
    # Initializations
    survivors = torch.ones(num_games).nonzero()               # Keeps track of ongoing games; everyone alive initially
    num_alive = len(survivors)                                # Actual batch size for each turn (initially num_games)
    prev_messages = torch.zeros(num_games, len_message).long() # Previous linguistic message for each game
    prev_proposals = torch.zeros(num_games, num_types).long()  # Previous proposal for each game

    # Play the games -------------------------------------------------------------------------
    for i_turn in range(max_N): # Loop through maximum possible number of turns for all games
        
        # Agent IDs
        id_1 = i_turn % 2    # Current player
        id_2 = int(not id_1) # Other player
        
        # Remove finished games (batch size decreases)
        N = N[survivors].view(num_alive, 1)
        pool = pool[survivors].view(num_alive, num_types)
        prev_messages = prev_messages[survivors].view(num_alive, len_message)
        prev_proposals = prev_proposals[survivors].view(num_alive, num_types)
        # Quantities different for each agent
        for j in range(num_agents):
            item_contexts[j] = item_contexts[j][survivors].view(num_alive,num_types*2)
            #rewards[j] = rewards[j][survivors].view(num_alive)
            #reward_losses[j] = reward_losses[j][survivors].view(num_alive)
        
        # Agent currently playing
        Agent = Agents[id_1]             
        item_context = item_contexts[id_1]
        
        # Play the game -------------------------------------------------------------
        term, prev_messages, proposals = Agent([item_context, prev_messages, prev_proposals], True, num_alive)
        
        # Compute reward loss (assumes 2 agents) ------------------------------------
        # Games terminated by the current agent (previous proposal accepted)
        
        finishers = term.squeeze().nonzero()          # squeeze is for getting rid of extra useless dimension that pops up for some reason
        num_finishers = len(finishers)
        
        if len(finishers) != 0:
            pool_12 = pool[finishers].view(num_finishers,num_types)
            
            share_2 = prev_proposals[finishers].view(num_finishers,num_types) # Share of other (previous proposal) 
            share_1 = pool_12 - share_2 # Share of this agent (remainder)
            
            # Zero reward if proposal exceeds pool
            invalid_batches = torch.sum(share_2>pool_12,1)>0
            share_2[invalid_batches] = 0
            share_1[invalid_batches] = 0
            
            utility_1 = item_contexts[id_1][:,num_types:] # Recall that item context is a concatenation of pool and utility
            utility_1 = utility_1[finishers].view(num_finishers,num_types)
            utility_2 = item_contexts[id_2][:,num_types:]
            utility_2 = utility_2[finishers].view(num_finishers,num_types)

            # Calculate reward and reward losses
            r1 = rewards_func_test(share_1, utility_1, pool_12)
            r2 = rewards_func_test(share_2, utility_2, pool_12)

            # Add rewards
            average_rewards[id_1][i_ep] += r1.squeeze().sum()
            average_rewards[id_2][i_ep] += r2.squeeze().sum()

        prev_proposals = proposals # Don't need previous proposals anymore so update it

        # Wrapping up the end of turn ------------------------------------------------
        # Remove finished games
        # In term and term_N, element = 1 means die
        term_N = (N <= (i_turn+1)).view(num_alive,1).long() # Last turn reached; i_turn + 1 since i_turn starts counting from 0
        # In survivors, element = 1 means live
        survivors = (term+term_N) == 0
        
        # Check if everyone's dead
        if survivors.sum() == 0: # If all games over, break episode
            # Average reward
            average_rewards[id_1][i_ep] /= num_games
            average_rewards[id_2][i_ep] /= num_games
            break;
            
        # Reshape
        survivors = ((term+term_N) == 0).nonzero()[:,0].view(-1,1)
        num_alive = len(survivors) # Number of survivors

        #print('i_turn = ' + str(i_turn))
        
    #print('i_ep = ' + str(i_ep))
    #print('----------------')
    
for i in range(num_agents):
    average_rewards[i] = average_rewards[i].mean()
    
print('End ------------------')

for i in range(num_agents):
    print('Reward for agent #' + str(i) + ': ' + str(average_rewards[i].numpy()))

Start ----------------
End ------------------
Reward for agent #0: 0.37278857827186584
Reward for agent #1: 0.01759309135377407


In [113]:
if average_rewards[i] != average_rewards[i]:
    print('blah')